In [31]:
import json
from uuid import uuid4
import pandas as pd
from self_driving_lab_demo import SelfDrivingLabDemoLiquid
from self_driving_lab_demo.demos.liquid import liquid_observe_sensor_data
from self_driving_lab_demo.utils.observe import get_paho_client

dummy = False


# def liquid_observe_sensor_data_2(R, Y, B, **kwargs):
#     # gain = 32 instead of 128 to avoid saturation of the sensor data
#     return liquid_observe_sensor_data(
#         R, Y, B, prerinse_power=1.0, prerinse_time=10.0, runtime=10.0, gain=32, **kwargs
#     )


def liquid_dummy_observe_sensor_data(R, Y, B, **kwargs):
    # return a fixed set of values (no interaction with real hardware)
    return {
        "utc_timestamp": 1671675884,
        "ch470": 21288,
        "ch410": 5835,
        "ch440": 65535,
        "sd_card_ready": False,
        "ch510": 21632,
        "ch550": 6760,
        "ch670": 8970,
        "utc_time_str": "2022-12-22 02:24:44",
        "onboard_temperature_K": 297.8537,
        "ch620": 2901,
        "ch583": 2057,
    }


observe_sensor_data_fn = (
    liquid_observe_sensor_data if not dummy else liquid_dummy_observe_sensor_data
)


with open("secrets.json", "r") as f:
    secrets = json.load(f)

pico_id = secrets["PICO_ID_3"]
sensor_topic = f"sdl-demo/picow/{pico_id}/as7341/"
paho_client = get_paho_client(sensor_topic) if not dummy else None

session_id = f"grid-random-bayes-liquid-{str(uuid4())[:4]}"


In [34]:
sdl = SelfDrivingLabDemoLiquid(
    autoload=True,
    max_power=0.5,
    observe_sensor_data_fn=observe_sensor_data_fn,
    observe_sensor_data_kwargs=dict(
        pico_id=secrets["PICO_ID_3"],
        session_id=session_id,
        client=paho_client,
        prerinse_power=1.0,
        prerinse_time=10.0,
        runtime=10.0,
        gain=32,
    ),
)

In [35]:
sdl.get_target_inputs()

{'R': 0.22770586934673454, 'Y': 0.015203423474249633, 'B': 0.17580269278419525}

In [23]:
sdl.target_results

{'utc_timestamp': 1671773259,
 'ch470': 5405,
 'ch410': 2967,
 'ch440': 21587,
 'sd_card_ready': False,
 'ch510': 11071,
 'ch550': 21534,
 'ch670': 6640,
 'utc_time_str': '2022-12-23 05:27:39',
 'onboard_temperature_K': 297.3855,
 'ch620': 16387,
 'ch583': 21938}

In [24]:
parameters = sdl.parameters[0:3]

In [25]:
from ax import optimize
from ax.utils.measurement.synthetic_functions import branin

def evaluation_function(parameters):
    data = sdl.evaluate(parameters)
    return data["frechet"]

best_parameters, values, experiment, model = optimize(
    parameters=parameters,
    evaluation_function=evaluation_function,
    total_trials = 27,
    minimize=True,
)

[INFO 12-22 22:28:35] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter R. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-22 22:28:35] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Y. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-22 22:28:35] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-22 22:28:35] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='R', parameter_type=FLOAT, range=[0.0, 0.5]), RangeParameter(name='Y', parameter_type=FLOAT, range=[0.0, 0.5]), RangeParameter(name='B', param